In [ ]:
!pip install tensorflow-text

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d aminesedkaoui/yelp-review-ratings

In [9]:
from sklearn import preprocessing
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import pandas as pd
import random
from keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from progressbar import progressbar

In [ ]:
!unzip "/content//yelp-review-ratings.zip" -d "/content/"

In [ ]:
df_rating = pd.read_csv('/content//yelp_review_ratings.csv')
df_rating.rename(columns = {'text': 'reviewText', 'stars': 'rating', 'business_id': 'itemID', 'user_id': 'userID'}, inplace = True)
df_rating = df_rating.drop(['Unnamed: 0'],axis=1)
df_rating.head()

In [6]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe[itemID].value_counts(sort=False):
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from float to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return user_item,df_user_item

In [10]:
def generateIDs(df,index,columns,min_index,max_index,min_column,max_column):
  users = df[index].unique()
  items = df[columns].unique()
  df_train = df.copy()

  New_User_IDs = random.sample(range(min_index,max_index),df[index].nunique())
  New_Item_IDs = random.sample(range(min_column,max_column),df[columns].nunique())
  i = 0
  for d in users :
    df_train[index].replace({d : New_User_IDs[i]}, inplace=True)
    i+=1

  i = 0
  for d in items :
    df_train[columns].replace({d : New_Item_IDs[i]}, inplace=True)
    i+=1
  return df_train

In [11]:
def preprocess_df(df,index,column,min_user_id,max_user_id,min_item_id,max_item_id):
  
  df_train = generateIDs(df,index,column,min_user_id,max_user_id,min_item_id,max_item_id)
  
  df_train['one']=df_train['rating'].apply(lambda x: 1 if x==1 else 0)
  df_train['two']=df_train['rating'].apply(lambda x: 1 if x==2 else 0)
  df_train['three']=df_train['rating'].apply(lambda x: 1 if x==3 else 0)
  df_train['four']=df_train['rating'].apply(lambda x: 1 if x==4 else 0)
  df_train['five']=df_train['rating'].apply(lambda x: 1 if x==5 else 0)
  df_train['six']=df_train['rating'].apply(lambda x: 1 if x==6 else 0)

  return df_train

In [12]:
from numpy.core.fromnumeric import size
#GMF

def Create_GMF(num_users,num_items,SIZE_):

  input_userID = layers.Input(shape=[1], name='user_ID')
  input_itemID = layers.Input(shape=[1], name='item_ID')

  user_emb_GMF = layers.Embedding(num_users, SIZE_, name='user_emb_GMF')(input_userID)
  item_emb_GMF = layers.Embedding(num_items, SIZE_, name='item_emb_GMF')(input_itemID)

  u_GMF = layers.Flatten()(user_emb_GMF)
  i_GMF = layers.Flatten()(item_emb_GMF)

  dot_layer = layers.Multiply()([u_GMF, i_GMF])

  out_layer = layers.Dense(6, activation='softmax', name='output')(dot_layer)

  GMF = tf.keras.Model([input_userID, input_itemID], out_layer)
  
  return GMF

def Train_GMF(model,X_train,Y_train,nbrEpochs):
  model.compile(optimizer = 'adam',
                    loss = tf.keras.losses.categorical_crossentropy ,
                    metrics=['accuracy'])
  model.fit(X_train,Y_train,epochs = nbrEpochs)
  return model

def user_item_ID_lists(userIDs,itemIDs):
  i = 0
  item_s = pd.Series()
  user_s = pd.Series()
  for user in progressbar(userIDs):
    temp = pd.Series(itemIDs)
    item_s = item_s.append(temp)
    temp = []
    temp = [user for item in itemIDs]
    temp = pd.Series(temp)
    user_s = user_s.append(temp)
  
  return user_s,item_s

def Fill_Cf_Matrix(model,userList,itemList,userIDs,itemIDs):

  prediction = model.predict([userList,itemList],verbose = 0)
  i = 0
  row = []
  matrix = []
  print("pred done")

  while i < userList.shape[0]:
    result = np.where(prediction[i] == np.amax(prediction[i]))[0][0] + 1
    row.append(result)
    if len(row) == itemIDs.size:
      matrix.append(row)
      row = []
       
    i += 1

  matrix_arr = np.array(matrix)
  dataframe = pd.DataFrame(matrix_arr, index = userIDs, columns = itemIDs)
 
  return dataframe


In [40]:
df_sample = df_rating[:1000]

In [41]:
df_train = preprocess_df(df_sample,'userID','itemID',5000,9999,20000,30000)

In [ ]:
df_train.shape

In [43]:
GMF = Create_GMF(df_train['userID'].max() + 1,df_train['itemID'].max() + 1,64)

In [44]:
df_train_sample = df_train
X_train, X_test,y_train,y_test= train_test_split(df_train_sample[['userID','itemID']],df_train_sample[['one','two','three','four','five','six']],stratify=df_train_sample[['one','two','three','four','five','six']])
#X_train, X_test,y_train,y_test= train_test_split(df_train[['User_ID','Item_ID']],df_train['rating'],stratify=df_train['rating'])

In [54]:
GMF_trained = Train_GMF(GMF,[X_train['userID'],X_train['itemID']],y_train,5)

Epoch 1/5
24/24 [==============================] - 1s 5ms/step - loss: 0.0646 - accuracy: 1.0000
Epoch 2/5
24/24 [==============================] - 0s 5ms/step - loss: 0.0514 - accuracy: 1.0000
Epoch 3/5
24/24 [==============================] - 0s 5ms/step - loss: 0.0413 - accuracy: 1.0000
Epoch 4/5
24/24 [==============================] - 0s 5ms/step - loss: 0.0335 - accuracy: 1.0000
Epoch 5/5
24/24 [==============================] - 0s 5ms/step - loss: 0.0275 - accuracy: 1.0000


In [55]:
user_item_matrix , df_mat = DatasetToUserItemDataFrame(df_train_sample,'userID','itemID','rating')

In [ ]:
GMF_trained.evaluate([X_test['userID'],X_test['itemID']],y_test)

In [57]:
New_User_IDs = df_train_sample.userID.unique()
New_Item_IDs = df_train_sample.itemID.unique()
New_User_IDs.size*New_Item_IDs.size

818062

In [58]:
user_s,item_s = user_item_ID_lists(New_User_IDs,New_Item_IDs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
100% (994 of 994) |######################| Elapsed Time: 0:00:05 Time:  0:00:05


In [59]:
df_mat_filled = Fill_Cf_Matrix(GMF_trained,user_s,item_s,New_User_IDs,New_Item_IDs)

pred done


In [60]:
df_mat_filled.head()

,27885,23256,25985,21644,24027,25702,28481,28893,22423,27585,...,25693,25156,22961,20797,21776,20404,23710,28375,29613,28998
6236,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
8696,5,5,5,5,5,4,4,5,4,5,...,2,5,5,5,5,5,5,4,5,3
9301,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
8128,5,5,5,5,5,3,5,1,5,5,...,4,5,5,5,5,3,5,3,5,1
7183,5,4,5,2,4,5,3,4,2,5,...,3,4,5,5,3,4,5,3,5,4


In [ ]:
df_mat_filled1 = pd.read_csv('/content/epinions_user_item_filled_75K_matrix_100k_leaning.csv')

In [61]:
df_mat.head()

,27885,23256,25985,21644,24027,25702,28481,28893,22423,27585,...,25693,25156,22961,20797,21776,20404,23710,28375,29613,28998
6236,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8696,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9301,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8128,0,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7183,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
df_mat_sample =df_mat_filled

In [ ]:
df_mat_sample.head()

In [ ]:
def to_binary(x):
  if (x > 0):
    x = 1
  return x

In [ ]:
def df_to_binary(df):
  for col in df.columns:
    df[col] = df[col].apply(to_binary)
  return df

In [65]:
#Autoencoder

encoder_input = layers.Input(shape=(df_mat_sample.shape[1]),name='user_item')
flat = layers.Flatten()(encoder_input)
#dropout = layers.Dropout(.2)(flat)
hid_encoder = layers.Dense(256,activation="relu")(flat)
# hid_encoder1 = layers.Dense(256,activation="relu")(hid_encoder)
# hid_encoder2 = layers.Dense(128,activation="relu")(hid_encoder1)
# hid_encoder3 = layers.Dense(10,activation="relu")(hid_encoder2)
encoder_output = layers.Dense(128,activation="relu")(hid_encoder)

decoder_input = layers.Dense(256,activation="relu")(encoder_output)
# decoder_hidden1 = layers.Dense(256,activation="relu")(decoder_input)
# decoder_hidden2 = layers.Dense(512,activation="relu")(decoder_hidden1)
# decoder_hidden3 = layers.Dense(25,activation="relu")(decoder_hidden2)
decoder_output = layers.Dense(df_mat_sample.shape[1],activation="relu")(decoder_input)

autoencoder = tf.keras.Model(inputs = encoder_input, outputs = decoder_output)

In [ ]:
def rmse (y_true,y_pred):
  y_pred = tf.cast(y_pred, tf.float32)
  y_true = tf.cast(y_true, tf.float32)
  rmse = y_true - y_pred
  rmse = tf.square(rmse)
  rmse = tf.math.reduce_mean(rmse)
  rmse = tf.math.sqrt(rmse)
  return rmse

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

In [66]:
autoencoder.compile(optimizer ='adam',
                    loss = tf.keras.losses.MeanAbsoluteError(),
                    metrics=['accuracy'])

In [ ]:
autoencoder.summary()

In [68]:
X_train,X_test,y_train,y_test = train_test_split(df_mat,df_mat_sample)

In [ ]:
X_train

In [ ]:
autoencoder.fit(X_train, y_train, epochs = 25)

In [73]:
autoencoder.evaluate(X_test,y_test)

8/8 [==============================] - 0s 3ms/step - loss: 0.8879 - accuracy: 0.0000e+00


[0.8879443407058716, 0.0]

In [ ]:
df_mat

In [ ]:
mat_pred = autoencoder.predict(df_mat)

102/102 [==============================] - 0s 2ms/step


In [ ]:
mat_pred

array([[5.0309396, 4.997768 , 5.486996 , ..., 5.0276556, 4.019191 ,
        5.0223575],
       [4.996603 , 4.997317 , 3.1208553, ..., 5.0028224, 3.7864075,
        4.999507 ],
       [4.9986353, 5.004016 , 2.3716743, ..., 5.0063057, 3.1234455,
        5.0051513],
       ...,
       [4.9948664, 4.9963155, 4.4013014, ..., 5.003981 , 4.028637 ,
        5.0001802],
       [4.9948664, 4.9963155, 4.4013014, ..., 5.003981 , 4.028637 ,
        5.0001802],
       [4.9948664, 4.9963155, 4.4013014, ..., 5.003981 , 4.028637 ,
        5.0001802]], dtype=float32)

In [ ]:
df_mat_pred = pd.DataFrame(mat_pred, index = df_mat_sample.index, columns = df_mat_sample.columns)

In [ ]:
df_mat_pred.max()

189722    6.472934
122571    6.492408
100211    6.818763
101960    6.571887
105337    6.085056
            ...   
137594    5.982568
198330    7.050071
136306    6.202019
197491    7.210152
124494    5.497391
Length: 1369, dtype: float32

In [ ]:
scaler = preprocessing.MinMaxScaler(feature_range=(1, 5))
d = scaler.fit_transform(df_mat_pred)
scaled_df = pd.DataFrame(d,index = df_mat_pred.index,columns = df_mat_pred.columns)
scaled_df

In [ ]:
for col in df_mat_pred.columns:
 df_mat_pred[col] = df_mat_pred[col].apply(lambda x: 5 if x> 5.5 else x)

In [ ]:
df_mat_pred

In [ ]:
res = np.where(df_mat_pred.iloc[:,:].values == 1 )
res[0].size